In [1]:
import urllib.request

def gather_input_data(url, sessionId, transform=lambda x: str(x, "utf-8").strip('\n')):
    request = urllib.request.Request(url)
    request.add_header("cookie", "session={}".format(sessionId)) # Source the data directly from AoC

    values = []
    with urllib.request.urlopen(request) as data:
        for line in data:
            values.append(transform(line))

    return values

def get_data(day, year=2023):
    with open('sessionID') as f:
        sessionId = f.readlines()[0].strip()
    url = "https://adventofcode.com/%d/day/%d/input"%(year,day)
    data = gather_input_data(url, sessionId)
    return data

from itertools import combinations
from z3 import Int, Ints, Solver

def intersect(hail1, hail2):
    (x1,y1,_),v1,(m1,b1) = hail1
    (x2,y2,_),v2,(m2,b2) = hail2
    if m1 == m2:
        out = False
    else:
        xi =(b2-b1) / (m1-m2)
        yi = m1*xi + b1
        t1 = (xi - x1) / v1[0]
        t2 = (xi - x2) / v2[0]
        if t1 < 0 or t2 < 0:
            out = False
        else:
            tmin = 200000000000000
            tmax = 400000000000000
            out = (tmin <= xi <= tmax) and (tmin <= yi <= tmax) 
    return out

def parse_data():
    hail = []
    for line in get_data(24):
        position_string,velocity_string = line.split('@')
        px,py,pz = map(int, position_string.split(','))
        vx,vy,vz = map(int, velocity_string.split(','))
        m = vy/vx
        b = py - m*px
        hail.append(((px,py,pz),(vx,vy,vz),(m,b)))
    return hail

In [2]:
hail = parse_data()
part1 = 0
for h1, h2 in combinations(hail, 2):
    if intersect(h1, h2):
        part1 += 1
print(part1)

pxr, pyr, pzr, vxr, vyr, vzr = Ints("pxr pyr pzr vxr vyr vzr")
s = Solver()
for k, ((pxh, pyh, pzh), (vxh, vyh, vzh), _) in enumerate(hail[:3]):
    tK = Int(f"t{k}")
    s.add(tK > 0)
    s.add(pxr + tK * vxr == pxh + tK * vxh)
    s.add(pyr + tK * vyr == pyh + tK * vyh)
    s.add(pzr + tK * vzr == pzh + tK * vzh)
s.check()
pxr = s.model()[pxr].as_long()
pyr = s.model()[pyr].as_long()
pzr = s.model()[pzr].as_long()
print(pxr + pyr + pzr) 


21785


In [3]:
import z3

554668916217145
